In [1]:
import sys
sys.path.append('/home/sensei/jupy-notebooks/Analytics/PorterFarms/')
print("============================================")
print("/  ScrubChannel is running.               /")
print("============================================")
import requests
from datetime import datetime, timedelta
import pytz
from slackclient import SlackClient
import json
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import configparser
import time

config = configparser.ConfigParser()
config.read("../../../analytics_secrets.ini")

_SLACK_TOKEN = config['slack']['token']
_CHIRPSTACK_USER = config['chirpstack']['user']
_CHIRPSTACK_PASS = config['chirpstack']['password']
_DB_HOST  = config['kanjidb']['dbhost']
_DB_PORT  = config['kanjidb']['dbport']
_DB_NAME  = config['kanjidb']['dbname']
_DB_USER  = config['kanjidb']['dbuser']
_DB_PASS  = config['kanjidb']['dbpass']
_USE_DROPBOX   = config['dropbox']['usedropbox']
_FALLBACK_IMAGE = config['dropbox']['fallbackimage']

_LOG_DEBUG = 0
_LOG_INFO  = 1
_LOG_ERROR = 2
_LOG_LEVEL = int(config['DEFAULT']['loglevel'])
def logger(level, message):
    if level >= _LOG_LEVEL:
      print(message)

logger(_LOG_DEBUG, "{} {} {} {} {}".format(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS))

import kanjiticketing as kt

conn = kt.getKanjiDbConnection(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS)
if conn is not None:
  print("Welcome to Jupyter Notebook.  You are connected to the Kanji database!")
else:
  print("You are not connected to the database.")

updatetemplate = "{\
    \"ok\": true, \
    \"channel\": \"C024BE91L\", \
    \"ts\": \"1401383885.000061\", \
    \"text\": \"Updated text you carefully authored\", \
    \"message\": { \
        \"text\": \"Updated text you carefully authored\", \
        \"user\": \"DGM50H51N\" \
    } }"

/  ScrubChannel is running.               /
Python version
3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]
Version info.
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)
Welcome to Jupyter Notebook.  You are connected to the Kanji database!


In [2]:
def updateMessageToSlack(slackchannel, ts, text):
    sc = SlackClient(_SLACK_TOKEN)
    response = sc.api_call("chat.update", channel=slackchannel, ts=ts, text=text, blocks=[])
    #print(blockmessage) 
    if not 'ok' in response or not response['ok']:
      print("Error posting message to Slack channel")
      print(blockmessage)
      print(response)
      return 'failed'
    else:
      print("Ok posting message to Slack channel")
      return 'ok'

In [3]:
STATUS_CLOSED   = 10003
STATUS_ARCHIVED = 10004
STATUS_SCRUBBED = 10005

now = datetime.now() + timedelta(hours = 4)

resolvedticketquery = "SELECT * FROM kanji_ticket WHERE status_id={}".format(STATUS_ARCHIVED)
#logger(_LOG_DEBUG, closedticketquery)
df = pd.read_sql(resolvedticketquery, conn)

logger(_LOG_DEBUG, "number of resolved tickets {}".format(len(df.index)))
for ind in df.index:
    logger(_LOG_DEBUG, ind)
    blockmessage = json.loads(updatetemplate)
    idticket = df['idticket'][ind]
    ticketts = df['ticketts'][ind]
    ticketchanneldbid = df['slackticketchannel_id'][ind]
    tickettypeid = df['type_id'][ind]
    ticketnodeid = df['node_id'][ind]
    logger(_LOG_INFO, "{} {}".format(idticket, ticketts))
    if idticket > 0:
      channelquery = "SELECT channelname, channelid FROM kanji_slackchannel WHERE idslackchannel={}".format(ticketchanneldbid)
      df2 = pd.read_sql(channelquery, conn)
      slackchannel = df2['channelid'][0]
      channelname = df2['channelname'][0]
      sc = SlackClient(_SLACK_TOKEN)
      response = sc.api_call("chat.delete", channel=slackchannel, ts=ticketts)
      logger(_LOG_INFO, "Deleting message ts={} from channel={}".format(ticketts, channelname))
      if not 'ok' in response or not response['ok']:
        print(response)
      kt.updateTicketStatus(conn, idticket, STATUS_SCRUBBED)
      time.sleep(1)
logger(_LOG_INFO, "ScrubChannel Done!")

ScrubChannel Done!
